In [31]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [32]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

In [33]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [34]:
# let's separate numeric columns and categorical columns 
num_cols = train.select_dtypes(include=np.number).columns.tolist()
num_cols.remove('id')
cat_cols = train.select_dtypes(include = ['object', 'string']).columns.tolist()

In [35]:
num_features = ['study_hours', 'class_attendance', 'sleep_hours']
cat_features = cat_cols   # your categorical columns

X = train[num_features + cat_features]
y = train['exam_score']

In [36]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_features)
    ]
)

In [37]:
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', LinearRegression())
])

In [38]:
score = cross_val_score(pipeline, X, y, cv=5, scoring='r2').mean()
print("Pipeline R2:", score)

Pipeline R2: 0.7789032591516565


In [41]:
pipeline.fit(X, y)

,steps,"[('preprocessing', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [42]:
X_test = test[num_features + cat_features]

In [43]:
test_predictions = pipeline.predict(X_test)

In [44]:
submission = pd.DataFrame({
    "id": test["id"],
    "exam_score": test_predictions
})

submission.to_csv("submission.csv", index=False)

In [45]:
submission.shape

(270000, 2)